In [1]:
%load_ext autoreload
%autoreload 2
    

# Running MD simulations using nff and ASE

This Jupyter Notebook shows how the `nff` package interfaces with the Atomistic Simulation Environment (ASE). We assume the user went through tutorial `01_training`, so we can load the pretrained models without having to train them again.

As before, importing the dependencies:

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from ase import Atoms
from ase.md.verlet import VelocityVerlet

from nff.md.nve import Dynamics
from nff.data import Dataset
from nff.train import load_model, evaluate
import nff.utils.constants as const
from ase import units
from nff.io import NeuralFF, AtomsBatch

## Loading the relevant data

We reload the dataset and create a `GraphLoader` as we did last time:

In [3]:
dataset = Dataset.from_file('data/dataset.pth.tar')


### Creating Atoms

As before, we can create an `Atoms` object from any element of the dataset. Let's take the first one, for simplicity:

In [4]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   )

### Creating the ASE calculator

Now we just have to load the ASE calculator from a pretrained model. One way of doing so is through the in-build `from_file` method. You just have to specify the folder where the model was trained and subsequently stored.

In [5]:
nff_ase = NeuralFF.from_file('sandbox/', device=0)

/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'nff.nn.models.schnet.SchNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve 

Assigning this calculator to `atoms` is easy:

In [6]:
atoms.set_calculator(nff_ase)

### Configuring the dynamics for the system

In this example, we will run an NVE dynamics simulation. We will use the default parameters there implemented to run a trajectory for an ethanol molecule. The parameters we will specify are the following:

* `T_init`: initial temperature of the simulation
* `time_step`: time step in femtoseconds
* `thermostat`: ASE integrator to use when performing the simulation
* `thermostat_params`: keyword arguments for ase.Integrator class, will be different case-by-case
* `steps`: number of steps to simulate
* `save_frequency`: how often (in steps) save the pose of the molecule in a file
* `nbr_list_update_freq`: how often (in steps) to update the neighbor list (not yet implemented)
* `thermo_filename`: output file for the thermodynamics log
* `traj_filename`: output file for the ASE trajectory file
* `skip`: number of initial frames to skip when recording the trajectory

In [7]:
md_params = {
    'T_init': 450,
    'time_step': 0.5,
#     'thermostat': NoseHoover,   # or Langevin or NPT or NVT or Thermodynamic Integration
#     'thermostat_params': {'timestep': 0.5 * units.fs, "temperature": 120.0 * units.kB,  "ttime": 20.0}
    'thermostat': VelocityVerlet,  
    'thermostat_params': {'timestep': 0.5 * units.fs},
    'steps': 200,
    'save_frequency': 10,
    'nbr_list_update_freq': 20,
    'thermo_filename': 'thermo.log',
    'traj_filename': 'atoms.traj',
    'skip': 0
}

In [8]:
nve = Dynamics(atoms, md_params)
nve.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]


/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/ase/atoms.py:921: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  np.VisibleDeprecationWarning)
/home/saxelrod/Repo/projects/master/NeuralForceField/nff/nn/utils.py:120: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  nbr_list = mask.nonzero()


0.0000           0.2305      -0.1916       0.4221   362.9

0.0050           0.2309      -0.2553       0.4862   418.0

0.0100           0.2319      -0.0995       0.3314   284.9

0.0150           0.2315      -0.1573       0.3887   334.2

0.0200           0.2307      -0.1688       0.3995   343.4

0.0250           0.2289      -0.2360       0.4649   399.6

0.0300           0.2297      -0.1202       0.3499   300.8

0.0350           0.2299      -0.1688       0.3987   342.7

0.0400           0.2305      -0.2976       0.5281   454.0

0.0450           0.2302      -0.2489       0.4791   411.8

0.0500           0.2283      -0.2310       0.4593   394.8

0.0550           0.2291      -0.4280       0.6571   564.8

0.0600           0.2296      -0.3177       0.5473   470.5

0.0650           0.2312      -0.0647       0.2959   254.3

0.0700           0.2326      -0.1137       0.3463   297.7

0.0750           0.2326      -0.0597       0.2924   251.3

0.0800           0.2314      -0.1302       0.3615   310.

### Models with directed neighbor lists

The default assumes that you're using SchNet, which uses an undirected neighbor list to save memory. If you're using Painn, DimeNet, or any model with directional information, you will need a directed neighbor list. If you don't specify this then you will get an error. 

While these models automatically make any neighbor list into a directed one when called, the same is not true of periodic boundary conditions. The dataest offsets from the PBC depend on whether the dataset neighbor list is directed or not.

In [9]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   )
nff_ase = NeuralFF.from_file('sandbox_painn/', device=0)
atoms.set_calculator(nff_ase)

try:
    nve = Dynamics(atoms, md_params)
    nve.run()
except Exception as e:
    print(e)

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
Painn needs a directed neighbor list


If you do then you will be fine!

In [10]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   directed=True
                   )
nff_ase = NeuralFF.from_file('sandbox_painn/', device=0)
atoms.set_calculator(nff_ase)

nve = Dynamics(atoms, md_params)
nve.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.5639      -0.1615       0.7254   623.6

0.0050           0.5644       0.0680       0.4963   426.6

0.0100           0.5643       0.2595       0.3048   262.0

0.0150           0.5635       0.0235       0.5400   464.2

0.0200           0.5623      -0.3615       0.9238   794.1

0.0250           0.5644       0.0448       0.5196   446.6

0.0300           0.5638       0.0454       0.5184   445.6

0.0350           0.5635      -0.1134       0.6769   581.9

0.0400           0.5635      -0.1133       0.6768   581.8

0.0450           0.5633       0.0204       0.5429   466.7

0.0500           0.5642       0.2131       0.3510   301.8

0.0550           0.5637      -0.0636       0.6274   539.3

0.0600           0.5640      -0.2444       0.8084   694.9

0.0650           0.5646      -0.0618       0.6264   538.4

0.0700           0.5635      -0.1222       0.6857   589.4

0.0750           0.5638       0.0751       0.4888   420.1


The dynamics conserved the energy. The temperature varied throughout the simulation, as expected.

# Additional properties

You can also specify the properties that you want the calculator to predict. The default is `energy` and `forces`, but you can also add `stress`.

However, if you run this for ethanol you will get an error, because it doesn't have any lattice vectors:

In [11]:
nff_ase.properties = ['energy', 'forces', 'stress']
atoms.set_calculator(nff_ase)

try:
    nve = Dynamics(atoms, md_params)
    nve.run()
except Exception as e:
    print(e)

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]      ---------------------- stress [GPa] -----------------------
You have 0 lattice vectors: volume not defined


If we make an artificial unit cell, it all works fine:

In [12]:
nff_ase.properties = ['energy', 'forces', 'stress']
atoms.set_calculator(nff_ase)
atoms.set_cell(2 * np.identity(3))
nve = Dynamics(atoms, md_params)
nve.run()



Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]      ---------------------- stress [GPa] -----------------------
0.0000           0.4056      -0.0772       0.4828   415.0    -64.503    -58.462     -6.680     36.943    -12.859     34.972

0.0050           0.4080      -0.1170       0.5250   451.3     -8.771     47.811     10.605    -27.834     21.692    -30.039

0.0100           0.4088      -0.0918       0.5006   430.3    -29.461    -44.988    -31.074     53.025     30.802     14.489

0.0150           0.4079      -0.0701       0.4780   410.8     85.145      3.028     39.714    -44.857     10.708    -34.974

0.0200           0.4067      -0.0911       0.4978   427.9     39.842    -10.329     16.883     21.105    -34.536     28.233

0.0250           0.4068      -0.0758       0.4826   414.9     89.950     47.283    -19.682    -28.243    -14.516    -12.615

0.0300           0.4049      -0.2810       0.6859   589.6     26.034     48.356      1.975     -0.968     -5.302     34.795

0.0

Since `stress` is an implemented property, we can get the stress on the atoms:

In [13]:
atoms.get_stress()

array([ 0.26794022,  0.19554217,  0.00357576,  0.17295389, -0.08392215,
        0.3378493 ], dtype=float32)

## Visualizing the trajectory

To visualize the trajectory in this Jupyter Notebook, you will have to install the package [nglview](https://github.com/arose/nglview).

In [14]:
import nglview as nv
from ase.io import Trajectory

Displaying the trajectory:

In [15]:
%matplotlib notebook

traj = Trajectory('atoms.traj')
nv.show_asetraj(traj)

NGLWidget(max_frame=20)

Looks like the atoms are still together. Visual inspection says that the trajectory is reasonable. Yay for `nff`!